In [100]:
import spacy
import requests
import os
from bs4 import BeautifulSoup
from collections import Counter
import matplotlib.pyplot as plt
import numpy
import pandas

In [57]:
def get_fileNames(category: str):
    file_names = []
    for files in os.listdir("Data/" + category):
        if files.endswith(".csv"):
            files = files[0:len(files)-4]
            file_names.append(files)
    return file_names

In [58]:
def get_text(article: str, url="https://en.wikipedia.org/wiki/"):
    soup = BeautifulSoup(requests.get(url + article).text, "lxml")
    tags = soup.find_all("p")
    text = ""

    for i in tags:
        text += i.text

    return text

def get_similarity(article1: str, article2: str):
    nlp = spacy.load("en_core_web_lg")
    doc1 = nlp(get_text(article1))
    doc2 = nlp(get_text(article2))
    return doc1.similarity(doc2)

In [63]:
files = get_fileNames("Music")

pairs = [(a,b) for a in files for b in files]

for article1, article2 in pairs:
    s = get_similarity(article1, article2)
    if s < 0.9:
        print(f"{get_similarity(article1, article2)} = {article1} <=> {article2}")

0.3050835700699651 = (Everything I Do) I Do It for You <=> ...Baby One More Time
0.39583430358225696 = (Everything I Do) I Do It for You <=> 7 Years
0.4701087086431092 = (Everything I Do) I Do It for You <=> All of Me
0.691691347195915 = (Everything I Do) I Do It for You <=> Bad Guy
0.345343483595698 = (Everything I Do) I Do It for You <=> Believe
0.4014793232583162 = (Everything I Do) I Do It for You <=> Believer
0.33363248847533183 = (Everything I Do) I Do It for You <=> Cheap Thrills
0.34551229528309196 = (Everything I Do) I Do It for You <=> Closer
0.7956107401737506 = (Everything I Do) I Do It for You <=> Dark Horse
0.5825210907172099 = (Everything I Do) I Do It for You <=> Diana
0.017125917165750983 = (Everything I Do) I Do It for You <=> Don't Let Me Down
0.3859037810777584 = (Everything I Do) I Do It for You <=> God's Plan
0.8863549337197822 = (Everything I Do) I Do It for You <=> Happy
0.3150325411086189 = (Everything I Do) I Do It for You <=> Hound Dog
0.054257648935984476 = 

KeyboardInterrupt: 

In [66]:
files = get_fileNames("Games")

pairs = [(a,b) for a in files for b in files]

for article1, article2 in pairs:
    s = get_similarity(article1, article2)
    if s < 0.9:
        print(f"{get_similarity(article1, article2)} = {article1} <=> {article2}")

0.7756746255250652 = Advance Wars <=> Borderlands
0.7656737847986517 = Advance Wars <=> God of War
0.516006886025122 = Advance Wars <=> Inside
0.5443503852624827 = Advance Wars <=> Journey
0.5906458947366489 = Advance Wars <=> Portal
0.5813941212962803 = Advance Wars <=> Rez
0.5651127283366908 = Advance Wars <=> The Walking Dead
0.688409126383116 = Angry Birds <=> Borderlands
0.7156183962112562 = Angry Birds <=> God of War


KeyboardInterrupt: 

In [67]:
files = get_fileNames("Literary")

pairs = [(a,b) for a in files for b in files]

for article1, article2 in pairs:
    s = get_similarity(article1, article2)
    if s < 0.9:
        print(f"{get_similarity(article1, article2)} = {article1} <=> {article2}")

0.49892508135015834 = A Confederacy of Dunces <=> American War
0.4407163725015864 = A Confederacy of Dunces <=> Beloved
0.37518132767304085 = A Confederacy of Dunces <=> Days Without End
0.786518106395682 = A Confederacy of Dunces <=> Eye of the Needle
0.46490339098456385 = A Confederacy of Dunces <=> Forever...
0.40641627775206296 = A Confederacy of Dunces <=> Golden Child
0.42965813967055505 = A Confederacy of Dunces <=> Habibi
0.8495860797410437 = A Confederacy of Dunces <=> Home Fire
0.5050068496877428 = A Confederacy of Dunces <=> Regeneration
0.5114811664697979 = A Confederacy of Dunces <=> Riders
0.45403408605125434 = A Confederacy of Dunces <=> Small Island
0.4162341534820867 = A Confederacy of Dunces <=> So Long, See You Tomorrow
0.5641483844859454 = A Confederacy of Dunces <=> The Outsiders
0.5647318163335133 = A Confederacy of Dunces <=> The Passion
0.6463960265884355 = A Confederacy of Dunces <=> The Prime of Miss Jean Brodie
0.5295342228664549 = A Confederacy of Dunces <=>

KeyboardInterrupt: 

In [68]:
files = get_fileNames("Movies")

pairs = [(a,b) for a in files for b in files]

for article1, article2 in pairs:
    s = get_similarity(article1, article2)
    if s < 0.9:
        print(f"{get_similarity(article1, article2)} = {article1} <=> {article2}")

0.3843139756042168 = 12 Years a Slave <=> Amour
0.4382388760646703 = 12 Years a Slave <=> Boyhood
0.64962355416197 = 12 Years a Slave <=> Caché
0.4276222767099958 = 12 Years a Slave <=> Carlos
0.41498195584760406 = 12 Years a Slave <=> Carol
0.6283998143490521 = 12 Years a Slave <=> Fish Tank
0.4207242755281497 = 12 Years a Slave <=> Her
0.42928433471913996 = 12 Years a Slave <=> Ida
0.35911836653393836 = 12 Years a Slave <=> Inside Out
0.5881089448953379 = 12 Years a Slave <=> Let the Right One In
0.5949319223746264 = 12 Years a Slave <=> Lost in Translation
0.6930090462029235 = 12 Years a Slave <=> Memento
0.8961097716638635 = 12 Years a Slave <=> Mulholland Drive
0.3520436868547946 = 12 Years a Slave <=> Oldboy
0.8600644019297502 = 12 Years a Slave <=> Ratatouille
0.5173269942079248 = 12 Years a Slave <=> Spotlight
0.4188224927825657 = 12 Years a Slave <=> Tabu
0.4537707247368009 = 12 Years a Slave <=> Ten
0.7216457515148066 = 12 Years a Slave <=> The Assassin
0.48733551088283317 = 

KeyboardInterrupt: 

In [110]:
nlp = spacy.load("en_core_web_lg")
def most_common_propn(article: str, count: int, pos="PROPN"):
    doc = nlp(get_text(article))

    propn = [token.text for token in doc if token.is_alpha and not token.is_stop and token.pos_ == pos]
    return Counter(propn).most_common(count)

In [115]:
files = get_fileNames("Games")

for file in files:
    print(f"{file}: {most_common_propn(file, 5)}")

Advance Wars: [('Advance', 34), ('Wars', 32), ('Nintendo', 21), ('Orange', 16), ('Star', 16)]
Angry Birds: [('Angry', 108), ('Birds', 101), ('Rovio', 41), ('June', 10), ('March', 8)]
Animal Crossing: [('Animal', 42), ('Crossing', 37), ('Nintendo', 27), ('New', 20), ('Horizons', 12)]
Assassin's Creed II: [('Assassin', 78), ('Creed', 62), ('Ezio', 50), ('II', 35), ('Ubisoft', 31)]
Battlefield 1942: [('Battlefield', 22), ('World', 8), ('War', 8), ('Game', 7), ('United', 6)]
Bayonetta: [('Bayonetta', 13), ('Nintendo', 9), ('Sega', 5), ('Kamiya', 3), ('October', 3)]
Beyond Good & Evil: [('Jade', 39), ('Evil', 26), ('Good', 25), ('Game', 19), ('Ancel', 18)]
BioShock Infinite: [('Infinite', 78), ('BioShock', 71), ('Booker', 52), ('Levine', 49), ('Elizabeth', 47)]
BioShock: [('BioShock', 81), ('Levine', 44), ('K', 40), ('Games', 34), ('Jack', 34)]
Bloodborne: [('Hunter', 45), ('Yharnam', 26), ('Bloodborne', 19), ('Souls', 17), ('Nightmare', 15)]
Borderlands: [('Borderland', 1)]
Braid: [('Japan

In [116]:
files = get_fileNames("Music")

for file in files:
    print(f"{file}: {most_common_propn(file, 5)}")

(Everything I Do) I Do It for You: [('Adams', 11), ('UK', 5), ('Chart', 5), ('Lange', 4), ('United', 4)]
...Baby One More Time: [('Time', 1)]
7 Rings: [('Grande', 51), ('U', 7), ('January', 6), ('Billboard', 6), ('Music', 5)]
7 Years: [('Years', 1)]
A Whiter Shade of Pale: [('Pale', 14), ('Whiter', 13), ('Shade', 13), ('Brooker', 10), ('Fisher', 9)]
All About That Bass: [('Trainor', 59), ('Bass', 47), ('Kadish', 13), ('Music', 13), ('Billboard', 11)]
All I Want for Christmas Is You: [('Christmas', 112), ('Carey', 91), ('December', 34), ('Hot', 25), ('Billboard', 13)]
All of Me: []
Bad Guy: [('Bad', 2), ('Guy', 1)]
Bad Romance: [('Gaga', 106), ('Fame', 14), ('Best', 14), ('Billboard', 11), ('Video', 9)]
Bei Mir Bist Du Schön: [('Secunda', 12), ('Bei', 8), ('Mir', 7), ('Jacobs', 7), ('Sisters', 6)]
Believe: []
Believer: []
Blurred Lines: [('Thicke', 60), ('Williams', 32), ('Blurred', 27), ('Gaye', 16), ('Music', 15)]
Call Me Maybe: [('Jepsen', 34), ('United', 12), ('Hot', 11), ('Billboar